<a href="https://colab.research.google.com/github/julianovale/project_trains/blob/master/Exemplo_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
from sympy import I, Matrix, symbols, Symbol, eye
from datetime import datetime
import numpy as np
import pandas as pd


In [63]:
'''
Rotas
'''
R1 = Matrix([[0,"L1_p3",0,0,0,0],[0,0,"L1_v1",0,0,0],[0,0,0,"L1_p4",0,0],[0,0,0,0,"L1_v3",0],[0,0,0,0,0,"L1_v4"],[0,0,0,0,0,0]])
R2 = Matrix([[0,"L2_p3",0,0,0,0],[0,0,"L2_v2",0,0,0],[0,0,0,"L2_p5",0,0],[0,0,0,0,"L2_v3",0],[0,0,0,0,0,"L2_v5"],[0,0,0,0,0,0]])
R3 = Matrix([[0,"L3_p3",0,0,0,0],[0,0,"L3_v5",0,0,0],[0,0,0,"L3_p1",0,0],[0,0,0,0,"L3_v3",0],[0,0,0,0,0,"L3_v1"],[0,0,0,0,0,0]])

In [64]:
'''
Seções de bloqueio
'''
T1 = Matrix([[0, "p1"],["v1", 0]])
T2 = Matrix([[0, "p2"],["v2", 0]])
T3 = Matrix([[0, "p3"],["v3", 0]])
T4 = Matrix([[0, "p4"],["v4", 0]])
T5 = Matrix([[0, "p5"],["v5", 0]])

In [65]:
def kronSum(A,B):
  m = np.size(A,1)
  n = np.size(B,1)
  A = np.kron(A, np.eye(n))
  B = np.kron(np.eye(m),B)
  return A + B

In [66]:
momento_inicio = datetime.now() 
'''
Algebra de rotas
'''
rotas = kronSum(R1,R2)
rotas = kronSum(rotas,R3)

'''
Algebra de seções 
'''
secoes = kronSum(T1,T2)
secoes = kronSum(secoes,T3)
secoes = kronSum(secoes,T4)
secoes = kronSum(secoes,T5)
'''
Algebra de sistema
'''
sistema = np.kron(rotas, secoes)
# calcula tempo de processamento
tempo_processamento = datetime.now() - momento_inicio

In [67]:
sistema = pd.DataFrame(data=sistema,index=list(range(1,np.size(sistema,0)+1)), columns=list(range(1,np.size(sistema,1)+1)))

In [68]:
sistema.shape

(6912, 6912)

In [69]:
print(tempo_processamento)

0:01:14.491733


In [70]:
sistema

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6873,6874,6875,6876,6877,6878,6879,6880,6881,6882,6883,6884,6885,6886,6887,6888,6889,6890,6891,6892,6893,6894,6895,6896,6897,6898,6899,6900,6901,6902,6903,6904,6905,6906,6907,6908,6909,6910,6911,6912
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0*L3_p3*p5,1.0*L3_p3*p4,0,1.0*L3_p3*p3,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0*L3_p3*v5,0,0,1.0*L3_p3*p4,0,1.0*L3_p3*p3,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0*L3_p3*v4,0,0,1.0*L3_p3*p5,0,0,1.0*L3_p3*p3,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0*L3_p3*v4,1.0*L3_p3*v5,0,0,0,0,1.0*L3_p3*p3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0*L3_p3*v3,0,0,0,0,1.0*L3_p3*p5,1.0*L3_p3*p4,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6908,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6909,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6910,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6911,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
sistema.loc[4740,4788]

1.0*L3_v1*p1

In [72]:
momento_inicio = datetime.now() 
colunas = ['denode', 'paranode', 'aresta']
grafo = pd.DataFrame(columns=colunas)
r = 1
c = 1
for j in range(np.size(sistema,0)):
  for i in range(np.size(sistema,0)):
    if sistema.loc[r,c]==0 and c < np.size(sistema,0):
      c += 1
    elif c < np.size(sistema,0):
      grafo.loc[len(grafo)+1] = (r, c, sistema.loc[r,c])  
      c += 1
    else:
      c = 1
      r += 1
tempo_processamento = datetime.now() - momento_inicio
print(tempo_processamento)

0:21:53.389261


In [73]:
grafo

,denode,paranode,aresta
1,1,34,1.0*L3_p3*p5
2,1,35,1.0*L3_p3*p4
3,1,37,1.0*L3_p3*p3
4,1,41,1.0*L3_p3*p2
5,1,49,1.0*L3_p3*p1
...,...,...,...
86381,6880,6896,1.0*L3_v1*v1
86382,6880,6904,1.0*L3_v1*v2
86383,6880,6908,1.0*L3_v1*v3
86384,6880,6910,1.0*L3_v1*v4


In [74]:
grafo['aresta'] = grafo['aresta'].astype('str')
grafo

,denode,paranode,aresta
1,1,34,1.0*L3_p3*p5
2,1,35,1.0*L3_p3*p4
3,1,37,1.0*L3_p3*p3
4,1,41,1.0*L3_p3*p2
5,1,49,1.0*L3_p3*p1
...,...,...,...
86381,6880,6896,1.0*L3_v1*v1
86382,6880,6904,1.0*L3_v1*v2
86383,6880,6908,1.0*L3_v1*v3
86384,6880,6910,1.0*L3_v1*v4


In [75]:
new = grafo["aresta"].str.split("*", n = -1, expand = True) 
grafo["aresta"]=new[1]
grafo["semaforo_secao"]=new[2]
new = grafo["aresta"].str.split("_", n = -1, expand = True) 
grafo["semaforo_trem"]=new[1]
grafo['coincide'] = np.where(grafo['semaforo_secao']==grafo['semaforo_trem'], True, False)
grafo

,denode,paranode,aresta,semaforo_secao,semaforo_trem,coincide
1,1,34,L3_p3,p5,p3,False
2,1,35,L3_p3,p4,p3,False
3,1,37,L3_p3,p3,p3,True
4,1,41,L3_p3,p2,p3,False
5,1,49,L3_p3,p1,p3,False
...,...,...,...,...,...,...
86381,6880,6896,L3_v1,v1,v1,True
86382,6880,6904,L3_v1,v2,v1,False
86383,6880,6908,L3_v1,v3,v1,False
86384,6880,6910,L3_v1,v4,v1,False


In [76]:
grafo = pd.DataFrame(data=grafo)

In [77]:
# Passo 1:
alcancavel = [1]
r = 1
for i in range(np.size(grafo,0)):
  de = grafo.loc[r]['denode']
  para = grafo.loc[r]['paranode']
  if (de in alcancavel):
    alcancavel.append(para)
    r = r + 1
  else:
    r = r + 1

In [78]:
alcancavel.sort()

In [79]:
grafo = grafo[grafo.denode.isin(alcancavel)]
grafo

,denode,paranode,aresta,semaforo_secao,semaforo_trem,coincide
1,1,34,L3_p3,p5,p3,False
2,1,35,L3_p3,p4,p3,False
3,1,37,L3_p3,p3,p3,True
4,1,41,L3_p3,p2,p3,False
5,1,49,L3_p3,p1,p3,False
...,...,...,...,...,...,...
86376,6878,6909,L3_v1,v5,v1,False
86377,6879,6895,L3_v1,v1,v1,True
86378,6879,6903,L3_v1,v2,v1,False
86379,6879,6907,L3_v1,v3,v1,False


In [80]:
grafo.reset_index(drop = True)
grafo.index = np.arange(1, len(grafo) + 1)
grafo

,denode,paranode,aresta,semaforo_secao,semaforo_trem,coincide
1,1,34,L3_p3,p5,p3,False
2,1,35,L3_p3,p4,p3,False
3,1,37,L3_p3,p3,p3,True
4,1,41,L3_p3,p2,p3,False
5,1,49,L3_p3,p1,p3,False
...,...,...,...,...,...,...
41861,6878,6909,L3_v1,v5,v1,False
41862,6879,6895,L3_v1,v1,v1,True
41863,6879,6903,L3_v1,v2,v1,False
41864,6879,6907,L3_v1,v3,v1,False


In [81]:
# Passo 2:
alcancavel = [6906]
r = np.size(grafo,0)
for i in range(np.size(grafo,0),1,-1):
  para = grafo.loc[r]['paranode']
  de = grafo.loc[r]['denode']
  if (para in alcancavel):
    alcancavel.append(de)
    r = r - 1
  else:
    r = r - 1

In [82]:
grafo = grafo[grafo.paranode.isin(alcancavel)]
grafo

,denode,paranode,aresta,semaforo_secao,semaforo_trem,coincide
1,1,34,L3_p3,p5,p3,False
2,1,35,L3_p3,p4,p3,False
3,1,37,L3_p3,p3,p3,True
4,1,41,L3_p3,p2,p3,False
5,1,49,L3_p3,p1,p3,False
...,...,...,...,...,...,...
41815,6858,6906,L3_v1,p1,v1,False
41834,6866,6906,L3_v1,p2,v1,False
41851,6873,6906,L3_v1,p5,v1,False
41856,6876,6906,L3_v1,v4,v1,False


In [83]:
grafo.reset_index(drop = True)
grafo.index = np.arange(1, len(grafo) + 1)
grafo

,denode,paranode,aresta,semaforo_secao,semaforo_trem,coincide
1,1,34,L3_p3,p5,p3,False
2,1,35,L3_p3,p4,p3,False
3,1,37,L3_p3,p3,p3,True
4,1,41,L3_p3,p2,p3,False
5,1,49,L3_p3,p1,p3,False
...,...,...,...,...,...,...
40556,6858,6906,L3_v1,p1,v1,False
40557,6866,6906,L3_v1,p2,v1,False
40558,6873,6906,L3_v1,p5,v1,False
40559,6876,6906,L3_v1,v4,v1,False


In [84]:
grafo.to_csv('grafo.csv', sep=";")
from google.colab import files
files.download('grafo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>